<a href="https://colab.research.google.com/github/SanjanaRitika/TextToCode_seq2seq/blob/main/improved_version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
# Import required libraries
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import math
import time
from datasets import load_dataset
from transformers import AutoTokenizer
from sklearn.model_selection import train_test_split

# Set device
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {DEVICE}")

Using device: cuda


In [17]:
# Load dataset
ds = load_dataset("Nan-Do/code-search-net-python")
print(ds)

DatasetDict({
    train: Dataset({
        features: ['repo', 'path', 'func_name', 'original_string', 'language', 'code', 'code_tokens', 'docstring', 'docstring_tokens', 'sha', 'url', 'partition', 'summary'],
        num_rows: 455243
    })
})


In [18]:
# Extract relevant columns
full_df = ds['train'].to_pandas()
df = full_df[['code', 'code_tokens', 'docstring', 'docstring_tokens']]

# Filter by sequence length constraints and sample
length_filter = (df['docstring_tokens'].map(len) <= 50) & (df['code_tokens'].map(len) <= 80)
filtered_data = df[length_filter]
sampled_data = filtered_data.sample(n=10000, random_state=42).reset_index(drop=True)

print(f"Sampled dataset shape: {sampled_data.shape}")
print("\nFirst few examples:")
print(sampled_data.head(3))

Sampled dataset shape: (10000, 4)

First few examples:
                                                code  \
0  def _lib(self, name, only_if_have=False):\n   ...   
1  def open(self):\n        """Opens an existing ...   
2  def hmac_sha1(self, key_handle, data, flags = ...   

                                         code_tokens  \
0  [def, _lib, (, self, ,, name, ,, only_if_have,...   
1  [def, open, (, self, ), :, try, :, self, ., gr...   
2  [def, hmac_sha1, (, self, ,, key_handle, ,, da...   

                                           docstring  \
0  Specify a linker library.\n\n        Example:\...   
1                           Opens an existing cache.   
2  Have the YubiHSM generate a HMAC SHA1 of 'data...   

                                    docstring_tokens  
0                   [Specify, a, linker, library, .]  
1                    [Opens, an, existing, cache, .]  
2  [Have, the, YubiHSM, generate, a, HMAC, SHA1, ...  


In [19]:
# Analyze token length statistics
stats_df = pd.DataFrame({
    'code_length': sampled_data['code_tokens'].apply(len),
    'docstring_length': sampled_data['docstring_tokens'].apply(len)
}).describe()

print("Dataset Statistics:")
print(stats_df)

Dataset Statistics:
        code_length  docstring_length
count  10000.000000        10000.0000
mean      48.047400           12.5361
std       15.613723            8.8588
min       20.000000            1.0000
25%       35.000000            7.0000
50%       46.000000           10.0000
75%       60.000000           15.0000
max       80.000000           50.0000


In [20]:
# Initialize CodeBERT tokenizer
tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base", add_prefix_space=True)
VOCAB_SIZE = tokenizer.vocab_size
PAD_IDX = tokenizer.pad_token_id

print(f"Vocabulary size: {VOCAB_SIZE}")
print(f"Padding index: {PAD_IDX}")

Vocabulary size: 50265
Padding index: 1


In [21]:
# Convert tokens to IDs
def tokenize_row(row):
    code_toks = [str(t) for t in row['code_tokens']]
    doc_toks = [str(t) for t in row['docstring_tokens']]

    code_ids = tokenizer(code_toks, is_split_into_words=True,
                         truncation=True, padding='max_length', max_length=128)['input_ids']
    doc_ids = tokenizer(doc_toks, is_split_into_words=True,
                        truncation=True, padding='max_length', max_length=64)['input_ids']

    return pd.Series({'code_ids': code_ids, 'docstring_ids': doc_ids})

tokenized_data = sampled_data.apply(tokenize_row, axis=1)
dataset = pd.concat([sampled_data, tokenized_data], axis=1)
print("\nTokenization complete!")


Tokenization complete!


In [22]:
# Split into train/validation/test sets
train_data, temp_data = train_test_split(dataset, test_size=0.40, random_state=42)
val_data, test_data = train_test_split(temp_data, test_size=0.50, random_state=42)

print(f"Train: {len(train_data)} | Validation: {len(val_data)} | Test: {len(test_data)}")

Train: 6000 | Validation: 2000 | Test: 2000


In [23]:
# Create DataLoaders
def create_dataloader(df, batch_size=64, shuffle=True):
    src_tensor = torch.tensor(df['code_ids'].tolist())
    trg_tensor = torch.tensor(df['docstring_ids'].tolist())
    dataset = TensorDataset(src_tensor, trg_tensor)
    return DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)

train_loader = create_dataloader(train_data)
val_loader = create_dataloader(val_data)
test_loader = create_dataloader(test_data, shuffle=False)

print("DataLoaders ready!")

DataLoaders ready!


In [24]:
class RNN_Encoder(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, pad_idx, n_layers=2, dropout=0.3):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, embed_dim, padding_idx=pad_idx)
        self.rnn = nn.RNN(
            embed_dim,
            hidden_dim,
            num_layers=n_layers,
            nonlinearity="tanh",
            batch_first=True,
            dropout=dropout if n_layers > 1 else 0.0
        )
        self.dropout = nn.Dropout(dropout)

    def forward(self, src):
        # src: [B, S]
        embedded = self.dropout(self.embed(src))  # [B,S,E]
        _, hidden = self.rnn(embedded)            # hidden: [L,B,H]
        return hidden


class RNN_Decoder(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, pad_idx, n_layers=2, dropout=0.3, tie_weights=True):
        super().__init__()
        self.vocab_size = vocab_size
        self.embed = nn.Embedding(vocab_size, embed_dim, padding_idx=pad_idx)
        self.rnn = nn.RNN(
            embed_dim,
            hidden_dim,
            num_layers=n_layers,
            nonlinearity="tanh",
            batch_first=True,
            dropout=dropout if n_layers > 1 else 0.0
        )
        self.dropout = nn.Dropout(dropout)

        # output projection
        self.out = nn.Linear(hidden_dim, vocab_size)

        # ✅ Weight tying (only if embed_dim == hidden_dim)
        if tie_weights:
            if embed_dim != hidden_dim:
                print("⚠️ tie_weights skipped because embed_dim != hidden_dim")
            else:
                self.out.weight = self.embed.weight  # tie

    def forward(self, token, hidden):
        # token: [B] (one token id per batch)  <-- simpler & standard
        token = token.unsqueeze(1)  # [B,1]
        embedded = self.dropout(self.embed(token))  # [B,1,E]
        output, hidden = self.rnn(embedded, hidden) # output: [B,1,H]
        prediction = self.out(output.squeeze(1))    # [B,V]
        return prediction, hidden


class VanillaSeq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, src, trg, teacher_ratio=0.5):
        # trg: [B,T]
        batch_size, trg_len = trg.shape
        vocab_size = self.decoder.vocab_size

        outputs = torch.zeros(batch_size, trg_len, vocab_size, device=self.device)

        hidden = self.encoder(src)   # [L,B,H]

        token = trg[:, 0]  # [B] (BOS)

        for t in range(1, trg_len):
            pred, hidden = self.decoder(token, hidden)
            outputs[:, t] = pred

            use_teacher = torch.rand(1).item() < teacher_ratio
            token = trg[:, t] if use_teacher else pred.argmax(1)

        return outputs

print("✅ Improved Vanilla RNN model defined")


✅ Improved Vanilla RNN model defined


In [25]:
# ----------------------------
# Better hyperparameters (Model 1: Vanilla RNN)
# ----------------------------
EMBED_DIM  = 256     # make equal to HIDDEN_DIM for weight tying
HIDDEN_DIM = 256
N_LAYERS   = 2
DROPOUT    = 0.3

N_EPOCHS = 10
CLIP = 1.0

# ----------------------------
# Initialize Model 1 (improved encoder/decoder)
# NOTE: this assumes you updated RNN_Encoder/RNN_Decoder to accept:
# (vocab_size, embed_dim, hidden_dim, pad_idx, n_layers, dropout, tie_weights)
# ----------------------------
encoder1 = RNN_Encoder(
    vocab_size=VOCAB_SIZE,
    embed_dim=EMBED_DIM,
    hidden_dim=HIDDEN_DIM,
    pad_idx=PAD_IDX,
    n_layers=N_LAYERS,
    dropout=DROPOUT
)

decoder1 = RNN_Decoder(
    vocab_size=VOCAB_SIZE,
    embed_dim=EMBED_DIM,
    hidden_dim=HIDDEN_DIM,
    pad_idx=PAD_IDX,
    n_layers=N_LAYERS,
    dropout=DROPOUT,
    tie_weights=True   # works because EMBED_DIM == HIDDEN_DIM
)

model1 = VanillaSeq2Seq(encoder1, decoder1, DEVICE).to(DEVICE)

# Multi-GPU (only if available)
if torch.cuda.device_count() > 1:
    print(f"Using {torch.cuda.device_count()} GPUs")
    model1 = nn.DataParallel(model1)

# ----------------------------
# Optimizer + Loss (better defaults)
# ----------------------------
optimizer1 = optim.AdamW(
    model1.parameters(),
    lr=3e-4,                 # smaller LR = more stable for seq2seq
    betas=(0.9, 0.98),
    weight_decay=1e-2
)

criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)


In [26]:
from torch.cuda.amp import autocast, GradScaler

scaler = GradScaler(enabled=(DEVICE.type == "cuda"))

# Training function (improved)
def train_model(model, loader, optimizer, criterion, clip, teacher_ratio=0.5):
    model.train()
    total_loss = 0
    total_acc = 0

    for src, trg in loader:
        src, trg = src.to(DEVICE), trg.to(DEVICE)
        optimizer.zero_grad(set_to_none=True)

        with autocast(enabled=(DEVICE.type == "cuda")):
            output = model(src, trg, teacher_ratio)

            # output: [B, T, V]
            # ignore BOS position
            output_flat = output[:, 1:].reshape(-1, output.shape[-1])
            trg_flat = trg[:, 1:].reshape(-1)

            loss = criterion(output_flat, trg_flat)

        # backward (AMP-safe)
        scaler.scale(loss).backward()

        # clip gradients
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        scaler.step(optimizer)
        scaler.update()

        total_loss += loss.item()

        # token accuracy (ignore PAD)
        preds = output[:, 1:].argmax(-1)              # [B, T-1]
        gold  = trg[:, 1:]                           # [B, T-1]
        mask  = gold != PAD_IDX
        correct = ((preds == gold) & mask).sum().item()
        total = mask.sum().item()
        total_acc += correct / (total + 1e-9)

    return total_loss / len(loader), total_acc / len(loader)


# Evaluation function (improved)
def eval_model(model, loader, criterion):
    model.eval()
    total_loss = 0
    total_acc = 0

    with torch.no_grad():
        for src, trg in loader:
            src, trg = src.to(DEVICE), trg.to(DEVICE)

            with autocast(enabled=(DEVICE.type == "cuda")):
                output = model(src, trg, 0)  # No teacher forcing

                output_flat = output[:, 1:].reshape(-1, output.shape[-1])
                trg_flat = trg[:, 1:].reshape(-1)

                loss = criterion(output_flat, trg_flat)

            total_loss += loss.item()

            preds = output[:, 1:].argmax(-1)
            gold  = trg[:, 1:]
            mask  = gold != PAD_IDX
            correct = ((preds == gold) & mask).sum().item()
            total = mask.sum().item()
            total_acc += correct / (total + 1e-9)

    return total_loss / len(loader), total_acc / len(loader)

print("✅ Improved training functions ready")


✅ Improved training functions ready


/tmp/ipython-input-2388924866.py:3: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler(enabled=(DEVICE.type == "cuda"))


In [27]:
best_val_loss_m1 = float("inf")
print("Training Vanilla RNN Seq2Seq...")

train_losses_m1 = []
val_losses_m1 = []
train_accs_m1 = []
val_accs_m1 = []

for epoch in range(N_EPOCHS):

    start_time = time.time()

    # teacher forcing schedule
    teacher_ratio = max(0.3, 0.7 - epoch * 0.05)

    train_loss, train_acc = train_model(
        model1,
        train_loader,
        optimizer1,
        criterion,
        CLIP,
        teacher_ratio=teacher_ratio
    )

    val_loss, val_acc = eval_model(
        model1,
        val_loader,
        criterion
    )

    train_losses_m1.append(train_loss)
    val_losses_m1.append(val_loss)
    train_accs_m1.append(train_acc)
    val_accs_m1.append(val_acc)

    epoch_time = time.time() - start_time

    if val_loss < best_val_loss_m1:
        best_val_loss_m1 = val_loss
        torch.save(model1.state_dict(), "vanilla_rnn.pt")

    print(
        "Epoch {:02d} | Time {:.0f}s | Train Loss {:.3f} | Val Loss {:.3f}".format(
            epoch + 1,
            epoch_time,
            train_loss,
            val_loss
        )
    )

print("Vanilla RNN training finished.")


Training Vanilla RNN Seq2Seq...


/tmp/ipython-input-2388924866.py:15: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=(DEVICE.type == "cuda")):
/tmp/ipython-input-2388924866.py:58: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=(DEVICE.type == "cuda")):


Epoch 01 | Time 72s | Train Loss 19.403 | Val Loss 12.681
Epoch 02 | Time 70s | Train Loss 13.138 | Val Loss 10.003
Epoch 03 | Time 68s | Train Loss 10.413 | Val Loss 8.846
Epoch 04 | Time 68s | Train Loss 9.134 | Val Loss 8.468
Epoch 05 | Time 68s | Train Loss 8.398 | Val Loss 8.029
Epoch 06 | Time 68s | Train Loss 7.852 | Val Loss 7.693
Epoch 07 | Time 68s | Train Loss 7.457 | Val Loss 7.473
Epoch 08 | Time 68s | Train Loss 7.194 | Val Loss 7.235
Epoch 09 | Time 68s | Train Loss 7.001 | Val Loss 7.135
Epoch 10 | Time 68s | Train Loss 6.847 | Val Loss 7.041
Vanilla RNN training finished.


In [ ]:
# Evaluate on test set
model1.load_state_dict(torch.load('vanilla_rnn.pt'))
test_loss = eval_model(model1, test_loader, criterion)
print(f'Vanilla RNN Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f}')